## Структурная классификация белков

Хотя аминокислотная последовательность однозначно определяет трёхмерную структуру белка, предсказание полной 3D-структуры является сложной задачей.

В этом задании вам предстоит обучить несколько моделей для более простой задачи: предсказания типа белковой укладки по аминокислотной последовательности:

<style>
td, th {
   border: none!important;
}
</style>
| Orthogonal bundle | Beta-sandwich | Alpha-beta roll |
| :-------: | :------: | :----: |
|<img src="../assets/images/orthogonal_bundle.png" width="300"/>|<img src="../assets/images/beta_sandwich.png" width="300"/>|<img src="../assets/images/alpha_beta_roll.png" width="300"/>|


Датасет получен на основе базы данных [CATH](https://www.cathdb.info/wiki?id=data:index), представляющей иерархическую и стандартизированную классификацию белковых доменов по типам укладки. В датасете сделана подвыборка 9 архитектур укладки белков, для каждой собрано 2k примеров небольшой длины последовательности.

Данные и метки классов лежат в `assets/datasets/protein_fold`

In [1]:
from pathlib import Path

import pandas as pd
import torch
from torch import Tensor
from torch.utils.data import Dataset, DataLoader

Класс датасета уже написан:

In [2]:
class CathSequencesDataset(Dataset):
    def __init__(self, subset_csv: Path, labels_csv: Path, add_cls_token: bool = False) -> None:
        # словарь: 20 аминокислот + 2 спец символа:
        # _ - pad token, добивает последовательность до нужной длины
        # ? — токен класса, он будет нужен для обучения трансформера
        vocab = "_?ACDEFGHIKLMNPQRSTVWY"
        df = pd.read_csv(subset_csv)
        labels = pd.read_csv(labels_csv)["architecture"]
        self.add_cls_token = add_cls_token
        self.vocab = {char: i for i, char in enumerate(vocab)}
        self.label_dict = {name: i for i, name in enumerate(labels)}
        self.sequences = [self._encode_sequence(s) for s in df["sequence"]]
        self.labels = [self.label_dict[label] for label in df["architecture"]]

    def __getitem__(self, index: int) -> tuple[list[int], int]:
        return self.sequences[index], self.labels[index]

    def __len__(self) -> int:
        return len(self.sequences)
    
    def _encode_sequence(self, sequence: str) -> list[int]:
        tokens = [self.vocab[char] for char in sequence]
        if self.add_cls_token:
            tokens = [1] + tokens
        return tokens

    @staticmethod
    def collate_fn(batch: list[tuple[list[int], int]]) -> tuple[Tensor, Tensor]:
        encoded, labels = zip(*batch)
        max_len = max(map(len, encoded))
        x = torch.zeros((len(encoded), max_len), dtype=int)
        for i, seq in enumerate(encoded):
            x[i, : len(seq)] = torch.tensor(seq)

        return x, torch.tensor(list(labels))

    def batch_decode(self, out_tokens: Tensor) -> list[str]:
        # декодирование всего батча токенов
        # сделано неточно, так как при получении токена окончания генерации (!)
        # остаток последовательности нужно отбросить
        inv_vocab = {v: k for k, v in self.vocab.items()}
        decoded_strings: list[str] = []
        for x in out_tokens:
            decoded_strings.append("".join([inv_vocab[i] for i in x.tolist() if i > 2]))

        return decoded_strings

In [3]:
rootdir = Path("../assets/datasets/protein_fold")
train_dataset = CathSequencesDataset(subset_csv=rootdir / "train.csv", labels_csv=rootdir / "labels.csv", add_cls_token=False)
tokens, label = train_dataset[0]
print("ID токенов:", tokens)
print("ID метки класса", label)

ID токенов: [14, 12, 19, 6, 11, 6, 14, 14, 10, 14, 10, 4, 18, 11, 12, 9, 17, 16, 18, 14, 5, 19, 18, 3, 19, 19, 19, 4, 19, 17, 4, 5, 4, 14, 5, 19, 10, 6, 13, 20, 21, 19, 4, 7, 19, 5, 19, 8, 13, 2, 10, 18, 10, 14, 16, 5, 5, 15, 21, 13, 2, 18, 21, 16, 19, 19, 17, 19, 11, 18, 19, 11, 8, 15, 4, 20, 11, 13, 7, 10, 5, 21, 10, 3, 10, 19, 17, 13, 10, 4, 11, 14, 2, 14, 9, 5, 10, 18, 9, 17]
ID метки класса 4


Последовательности в датасете имеют различную длину, поэтому для их объединения в один батч в `DataLoader(..., collate_fn=)` нужно будет передать фукнцию, которая правильно упаковывает список наблюдений в тензоры. Подходящая функция реализована в методе `CathSequencesDataset.collate_fn`

Посмотрим на мини-батч наблюдений:

In [4]:

train_dataset = CathSequencesDataset(subset_csv=rootdir / "train.csv", labels_csv=rootdir / "labels.csv", add_cls_token=False)
# sequences, labels = train_dataset.collate_fn([train_dataset[i] for i in range(4)])
train_loader = DataLoader(train_dataset, batch_size=4, collate_fn=train_dataset.collate_fn)
sequences, labels = next(iter(train_loader))
print("Последовательности:", sequences.shape)
print("Метки классов:", labels.shape)
sequences

Последовательности: torch.Size([4, 129])
Метки классов: torch.Size([4])


tensor([[14, 12, 19,  6, 11,  6, 14, 14, 10, 14, 10,  4, 18, 11, 12,  9, 17, 16,
         18, 14,  5, 19, 18,  3, 19, 19, 19,  4, 19, 17,  4,  5,  4, 14,  5, 19,
         10,  6, 13, 20, 21, 19,  4,  7, 19,  5, 19,  8, 13,  2, 10, 18, 10, 14,
         16,  5,  5, 15, 21, 13,  2, 18, 21, 16, 19, 19, 17, 19, 11, 18, 19, 11,
          8, 15,  4, 20, 11, 13,  7, 10,  5, 21, 10,  3, 10, 19, 17, 13, 10,  4,
         11, 14,  2, 14,  9,  5, 10, 18,  9, 17,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0],
        [ 7, 14, 11,  7, 17, 14,  5,  6, 18, 17, 12, 10, 14,  9, 13,  5,  4, 11,
          4,  5,  7,  9, 12, 19, 19, 21, 10, 16, 13,  9,  2,  7, 17,  7,  3, 15,
         11, 18,  6, 20,  5,  2, 17,  5, 16, 18,  9, 16, 17,  5,  2,  5,  4, 17,
         21,  8,  6, 17, 17,  2, 10, 12, 18,  2, 18,  6, 11, 17, 10, 10, 15,  5,
         19, 13, 12, 17,  4, 17,  2, 11,  4,  3, 19, 16,  4,  5,  2,  9, 13, 10,
      

### Задание 1 (5 баллов). Обучение классификатора на основе `nn.GRU`

Реализуйте классификатор последовательностей на основе GRU.
Ваш модуль будет иметь три нейросетевых блока:
1. Получение эмбеддингов для токенов
2. Получение скрытых состояний из рекуррентной сети
3. Классификатор последовательности на основе последнего скрытого состояния

В этом задании добейтесь точности классификации на валидационной выборке > 50%, для этого должно быть достаточно обучения в 15-20 эпох.

In [ ]:
from torch import Tensor, nn


class GRUClassifier(nn.Module):
    """Модель для классификации"""

    def __init__(self, vocab_size: int, hidden_dim: int, n_classes: int) -> None:
        super().__init__()
        # эмбеддинги для аминокислот
        self.embed = nn.Embedding(vocab_size, hidden_dim, padding_idx=0)
        # рекуррентный модуль
        self.rnn = nn.GRU(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=1,
            batch_first=True,  # обратите внимание на этот параметр! Это частый источник ошибок при работе с рекуррентными сетями
        )
        # голова модели
        self.classifier = nn.Linear(hidden_dim, n_classes)

    def forward(self, tokens: Tensor) -> Tensor:
        # B — размер батча
        # L — длина последовательности
        # H — размерность эмбеддингов
        # C — число классов
        # V — размер словаря токенов

        # получаем эмбеддинги токенов: B x L -> B x L x H
        x = ...
        # обрабатываем последовательность токенов с помощью RNN: B x L x H -> B x L x H
        ...
        # извлекаем последние скрытые состояния: B x L x H -> B x H
        last_state = ...

        # используем их для классификации последовательностей: B x H -> B x C
        logits = ...


### Задание 2 (5 баллов). Обучение классификатора на основе TransformerEncoder

Реализуйте классификатор последовательностей на основе блоков трансформера, приведённых в [notebooks/transformer_attention.ipynb](../notebooks/transformer_attention.ipynb).

Ваш модуль будет иметь три нейросетевых блока:
1. Получение начальных эмбеддингов для токенов и позиций
2. Преобразование эмбеддингов последовательностью блоков трансформера
3. Классификатор на основе финального эмбеддинга для токена <CLS>, который находится в начале каждой последовательности (не забудьте его включить в датасете!)

В этом задании добейтесь точности классификации на валидационной выборке > 60%, для этого должно быть достаточно обучения в 15-20 эпох.

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size: int, hidden_size: int, n_layers: int, n_classes: int = 9, max_position: int = 300):
        super().__init__()
        self.embeds = ...  # эмбеддинги токенов + позиций
        self.trunc = ...  # последовательность SelfAttention модулей
        self.classifier = ...  # классификатор

    def forward(self, x: Tensor) -> Tensor:
        ...

### Задание 3 (2 балла). Pre-norm vs Post-norm

Измените порядок следования механизма внимания, MLP и нормализаций на схему pre-normalized активаций (см. слайд 77 **Постпроцессинг токенов: нормализация и MLP** в [slides/dl_architectures.pdf](../slides/dl_architectures.pdf)).

Запустите эксперимент с вашей лучшей конфигурацией модели из задания 2, заменив только определение `TransformerLayer`.

Изменилась ли точность или динамика обучения?